In [1]:
import os
import glob
import tensorflow as tf
from tqdm import tqdm

# --- 설정 ---
# ▼▼▼ 검사할 데이터 폴더의 경로를 입력하세요 ▼▼▼
DATA_PATH = '../kfood'

# --- 메인 실행 로직 ---
print(f"TensorFlow 기반 데이터셋 무결성 검사를 시작합니다...")
print(f"대상 경로: {os.path.abspath(DATA_PATH)}")

all_files = glob.glob(os.path.join(DATA_PATH, '**', '*'), recursive=True)
all_files = [f for f in all_files if os.path.isfile(f)]

# 문제가 있는 파일 경로를 저장할 리스트
problematic_files = []

if not all_files:
    print("검사할 파일이 없습니다.")
else:
    for file_path in tqdm(all_files, desc="파일 검사 중"):
        try:
            # 1. TensorFlow로 파일을 바이트 단위로 읽기
            img_bytes = tf.io.read_file(file_path)
            
            # 2. TensorFlow로 이미지 디코딩 시도 (학습 시와 동일한 방식)
            # 이 부분이 실패하면 학습 시에도 실패합니다.
            tf.io.decode_image(img_bytes)

        except tf.errors.InvalidArgumentError as e:
            # TensorFlow가 이미지로 인식하지 못하는 경우
            if "Unknown image file format" in str(e):
                problematic_files.append({
                    "path": file_path,
                    "reason": "TensorFlow가 인식할 수 없는 이미지 형식 또는 손상된 파일입니다."
                })
        except Exception as e:
            # 기타 예외 처리
            problematic_files.append({
                "path": file_path,
                "reason": f"알 수 없는 오류: {str(e)}"
            })

# --- 결과 출력 ---
print("\n" + "="*50)
if not problematic_files:
    print("✅ 모든 파일을 성공적으로 확인했습니다. TensorFlow 호환성 문제가 발견되지 않았습니다.")
else:
    print(f"⚠️ 총 {len(problematic_files)}개의 문제 파일을 찾았습니다.")
    print("--- 문제 파일 목록 ---")
    for item in problematic_files:
        print(f"파일: {item['path']}")
        print(f"  원인: {item['reason']}")
        print("-" * 20)
    
    print("\n위 파일들을 삭제하거나 수정 후 다시 시도해 주세요.")
print("="*50)

2025-08-23 18:41:11.180315: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow 기반 데이터셋 무결성 검사를 시작합니다...
대상 경로: /workspace/data/kfood


파일 검사 중:   1%|▏         | 1945/150161 [00:15<19:43, 125.25it/s]


KeyboardInterrupt: 

In [2]:
import os
import glob
from PIL import Image
from tqdm import tqdm

# 검사할 데이터 폴더 경로
DATA_PATH = '../kfood'

print(f"'{os.path.abspath(DATA_PATH)}' 경로에서 확장자와 실제 형식이 다른 파일을 검색합니다...")

image_files = glob.glob(os.path.join(DATA_PATH, '**', '*'), recursive=True)
image_files = [f for f in image_files if os.path.isfile(f)]

mismatched_files = []

for file_path in tqdm(image_files, desc="파일 형식 확인 중"):
    try:
        with Image.open(file_path) as img:
            # .format 속성은 파일 내용물을 기반으로 실제 형식을 알려줍니다.
            # JPEG 파일의 경우 'JPEG'를 반환합니다.
            real_format = img.format
            if real_format != 'JPEG':
                mismatched_files.append((file_path, real_format))
    except Exception:
        # 손상된 파일은 무시
        continue

# --- 결과 출력 ---
print("\n" + "="*50)
if not mismatched_files:
    print("✅ 모든 .jpg/.jpeg 파일의 실제 형식이 일치합니다.")
else:
    print(f"⚠️ 총 {len(mismatched_files)}개의 '위장' 파일을 찾았습니다.")
    print("--- 파일 목록 ---")
    for f_path, f_format in mismatched_files:
        print(f"파일: {f_path} (실제 형식: {f_format})")
print("="*50)

'/workspace/data/kfood' 경로에서 확장자와 실제 형식이 다른 파일을 검색합니다...


파일 형식 확인 중:  10%|▉         | 14882/150161 [00:21<02:51, 787.16it/s]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))
파일 형식 확인 중: 100%|██████████| 150161/150161 [00:59<00:00, 2525.75it/s] 


✅ 모든 .jpg/.jpeg 파일의 실제 형식이 일치합니다.


In [5]:
import os
import glob
from PIL import Image, ImageFile
from tqdm import tqdm
import numpy as np

# Pillow가 불완전한 이미지를 로드할 때 오류를 발생시키도록 설정
ImageFile.LOAD_TRUNCATED_IMAGES = False

def clean_and_validate_dataset(data_path, min_size=(64, 64), std_dev_threshold=10):
    """
    주어진 경로의 이미지 데이터셋을 정리하고 검증하는 함수.
    
    Args:
        data_path (str): 이미지 데이터셋의 루트 경로.
        min_size (tuple): 유효한 이미지의 최소 가로, 세로 크기 (기본값: 64, 64).
        std_dev_threshold (int): 이미지의 픽셀 표준편차 임계값. 이 값보다 낮으면 단색으로 간주하여 제외 (기본값: 10).
    """
    print("✅ 이미지 데이터셋 정리 및 검증을 시작합니다...")
    print(f"대상 경로: {os.path.abspath(data_path)}")

    # 1. 모든 이미지 파일 목록 생성
    image_files = []

    supported_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif', '*.bmp', '*.tiff', '*.webp']
    for ext in supported_extensions:
        image_files.extend(glob.glob(os.path.join(data_path, '**', ext), recursive=True))

    
    total_files = len(image_files)
    if total_files == 0:
        print("🔍 처리할 이미지 파일이 없습니다.")
        return

    print(f"총 {total_files}개의 이미지 파일을 검증 및 처리합니다...")

    invalid_count = 0
    converted_count = 0
    
    for file_path in tqdm(image_files, desc="이미지 처리 중"):
        try:
            # 파일 로드 및 유효성 검증
            with Image.open(file_path) as img:
                img.verify() # 파일 구조 및 형식 검증

            # 2. 이미지 품질 및 크기 검증
            with Image.open(file_path) as img:
                # 최소 크기 확인
                if img.width < min_size[0] or img.height < min_size[1]:
                    tqdm.write(f"❌ 크기 미달 파일 제외: {file_path}")
                    os.remove(file_path)
                    invalid_count += 1
                    continue

                # 단색 이미지 확인
                pixels = np.array(img.convert('L'))
                if pixels.std() < std_dev_threshold:
                    tqdm.write(f"❌ 단색 이미지 제외: {file_path}")
                    os.remove(file_path)
                    invalid_count += 1
                    continue
            
            # 3. 파일 형식 통일 (JPEG)
            with Image.open(file_path) as img:
                if img.format != 'JPEG' or os.path.splitext(file_path)[1].lower() not in ['.jpg', '.jpeg']:
                    base_name, _ = os.path.splitext(file_path)
                    new_file_path = base_name + '.jpg'
                    
                    # 중복 이름 처리
                    counter = 1
                    while os.path.exists(new_file_path) and file_path != new_file_path:
                        new_file_path = f"{base_name}_{counter}.jpg"
                        counter += 1
                    
                    img.convert('RGB').save(new_file_path, 'JPEG', quality=95)
                    
                    if file_path != new_file_path:
                        os.remove(file_path)
                    
                    converted_count += 1

        except Exception as e:
            tqdm.write(f"❌ 손상된 파일 발견, 삭제: {file_path} - 오류: {e}")
            try:
                os.remove(file_path)
            except OSError:
                pass # 이미 파일이 없으면 무시
            invalid_count += 1

    print("\n" + "="*50)
    print("✅ 데이터셋 정리 및 검증이 완료되었습니다.")
    print(f"  - 총 처리된 파일: {total_files}개")
    print(f"  - JPG로 변환/수정된 파일: {converted_count}개")
    print(f"  - 삭제된 손상/오류 파일: {invalid_count}개")
    print("="*50)

# --- 실행 ---
if __name__ == "__main__":

    DATA_PATH = '../kfood' 
    clean_and_validate_dataset(DATA_PATH)

✅ 이미지 데이터셋 정리 및 검증을 시작합니다...
대상 경로: /workspace/data/kfood
총 147944개의 이미지 파일을 검증 및 처리합니다...


이미지 처리 중:   1%|▏         | 2022/147944 [00:28<34:42, 70.08it/s]  


KeyboardInterrupt: 

In [2]:
import warnings
from PIL import Image, TiffImagePlugin
import os
from tqdm import tqdm

def fix_jpeg_sos_parameters(root_folder):
    for dirpath, dirnames, filenames in os.walk(root_folder):
        image_files = [f for f in filenames if f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.tif') or f.endswith('.tiff')]
        for filename in tqdm(image_files, desc=f'Processing {dirpath}', leave=False):
            image_path = os.path.join(dirpath, filename)
            try:
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always")
                    with Image.open(image_path) as img:
                        img.save(image_path, "JPEG")

                    # 경고가 발생했으면 출력
                    for warn in w:
                        if issubclass(warn.category, UserWarning):
                            print(f"Warning for file {filename}: {warn.message}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

image_directory = "../kfood"
fix_jpeg_sos_parameters(image_directory)
